In [1]:
from spygrt.stream import Camera, DualCamera
from spygrt.tracker import Tracker
from spygrt.calibration import Calibrator
import open3d as o3d
import numpy as np
import pyrealsense2 as rs2

In [8]:
cam = []
ctx = rs2.context()
for dev in ctx.query_devices():
    cam.append(Camera(dev))

In [9]:
d = DualCamera(cam[0],cam[1])

In [10]:
d.start_stream()

In [11]:
d.get_frames()

((<pyrealsense2.frame Z16 #1864 @1682924536468.011719>,
  <pyrealsense2.frame RGB8 #1897 @1682924536468.348877>),
 (<pyrealsense2.frame Z16 #1875 @1682924536451.921143>,
  <pyrealsense2.frame RGB8 #1877 @1682924536452.258301>))

In [12]:
d.timestamp

1682924536459.9663

In [13]:
d.compute_pcd()

PointCloud on CUDA:0 [1843200 points (Float32)].
Attributes: colors (dtype = UInt8, shape = {1843200, 3}).

In [14]:
d.pose

(array([[-0.75023073,  0.47146061, -0.51771986,  0.39977682],
        [ 0.69985676,  0.52932656, -0.5321362 ,  0.42830384],
        [ 0.02256957, -0.74208724, -0.70848596,  0.49742207],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[-0.78175622, -0.49430156,  0.47189036, -0.43175048],
        [-0.67748666,  0.4665783 , -0.63361961,  0.4585802 ],
        [ 0.08959016, -0.78493822, -0.67379743,  0.50959456],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))

In [15]:
calibs = (Calibrator(d.stream1), Calibrator(d.stream2))

In [16]:
for i in range (100):
    calibs[0].find_corners3d(d.stream1.get_frames('rs'))
    calibs[1].find_corners3d(d.stream2.get_frames('rs'))

In [17]:
calibs[0].corners_scale()
calibs[1].corners_scale()
calibs[0].align_cameras(calibs[1])
calibs[1].align_to_board(cal2 = calibs[0])

In [20]:
for cal in calibs:
    cal.write_cal()

In [23]:
d.load_calibration()

(array([[ 0.81737643,  0.29225767, -0.49464679,  0.38554969],
        [ 0.57185096, -0.49688792,  0.65137011, -0.57059616],
        [-0.05546623, -0.8160159 , -0.57379061,  0.49654818],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.87549871, -0.32919836,  0.37581861, -0.41491762],
        [-0.49947402, -0.59447068,  0.64283699, -0.58003139],
        [ 0.01169839, -0.74454159, -0.6794337 ,  0.48179466],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))

In [22]:
d.stream1.load_calibration()
d.stream2.load_calibration()

array([[ 0.87549871, -0.32919836,  0.37581861, -0.41491762],
       [-0.49947402, -0.59447068,  0.64283699, -0.58003139],
       [ 0.01169839, -0.74454159, -0.6794337 ,  0.48179466],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [24]:
pcd = d.compute_pcd()

In [25]:
o3d.visualization.draw_geometries([pcd.to_legacy()])

999